<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
补充代码来自 <a href="http://mng.bz/orYv">从零构建大型语言模型</a> 一书，作者为 <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>代码仓库: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

# 理解嵌入层与线性层之间的区别

- 在 PyTorch 中，嵌入层完成的工作与执行矩阵乘法的线性层相同；我们使用嵌入层的原因是计算效率
- 我们将通过 PyTorch 中的代码示例逐步查看这种关系

In [1]:
import torch

print("PyTorch version:", torch.__version__)

PyTorch version: 2.2.2


## 使用 nn.Embedding

In [2]:
# 假设我们有以下 3 个训练样本，表示 LLM 场景中的 token ID
idx = torch.tensor([2, 3, 1])

# 通过获取最大的 token ID + 1 来确定嵌入矩阵的行数
# 如果最大的 token ID 是 3，那么我们需要 4 行，分别对应可能的 token ID 0, 1, 2, 3
num_idx = max(idx) + 1

# 设定嵌入维度作为超参数
out_dim = 5

- 让我们实现一个简单的嵌入层：

In [3]:
# 使用随机种子确保结果可复现，因为嵌入层中的权重是用随机小值初始化的
torch.manual_seed(123)

# 创建嵌入层，嵌入矩阵的行数为 num_idx，嵌入维度为 out_dim
embedding = torch.nn.Embedding(num_idx, out_dim)

我们可以选择查看嵌入权重：

In [4]:
embedding.weight

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.3035, -0.5880,  1.5810],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015],
        [ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953]], requires_grad=True)

- 然后，我们可以使用嵌入层来获取ID为1的训练样本的向量表示：

In [5]:
embedding(torch.tensor([1]))

tensor([[ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]],
       grad_fn=<EmbeddingBackward0>)

- 下面是一个关于底层发生的事情的可视化：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/1.png" width="400px">

- 同样，我们可以使用嵌入层来获取ID为2的训练样本的向量表示：

In [6]:
embedding(torch.tensor([2]))

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315]],
       grad_fn=<EmbeddingBackward0>)

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/2.png" width="400px">

- 现在，让我们转换之前定义的所有训练样本：

In [7]:
idx = torch.tensor([2, 3, 1])
embedding(idx)

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]],
       grad_fn=<EmbeddingBackward0>)

- 在底层，它仍然是相同的查找概念：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/3.png" width="450px">

## 使用 nn.Linear

- 现在，我们将演示上述嵌入层在 PyTorch 中与对独热编码表示应用的 `nn.Linear` 层完成的是完全相同的工作。
- 首先，让我们将 token ID 转换为独热表示：

In [10]:
onehot = torch.nn.functional.one_hot(idx)
onehot

tensor([[0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 1, 0, 0]])

- 接下来，我们初始化一个 `Linear` 层，该层执行矩阵乘法 $X W^\top$：

In [11]:
torch.manual_seed(123)
linear = torch.nn.Linear(num_idx, out_dim, bias=False)
linear.weight

Parameter containing:
tensor([[-0.2039,  0.0166, -0.2483,  0.1886],
        [-0.4260,  0.3665, -0.3634, -0.3975],
        [-0.3159,  0.2264, -0.1847,  0.1871],
        [-0.4244, -0.3034, -0.1836, -0.0983],
        [-0.3814,  0.3274, -0.1179,  0.1605]], requires_grad=True)

- 注意，PyTorch 中的线性层也是用小的随机权重初始化的；为了直接将其与上面的 `Embedding` 层进行比较，我们必须使用相同的随机权重，因此在这里重新分配它们：

In [10]:
linear.weight = torch.nn.Parameter(embedding.weight.T)

- 现在我们可以在线性层上使用输入的一热编码表示：

In [12]:
linear(onehot.float())

tensor([[-0.2483, -0.3634, -0.1847, -0.1836, -0.1179],
        [ 0.1886, -0.3975,  0.1871, -0.0983,  0.1605],
        [ 0.0166,  0.3665,  0.2264, -0.3034,  0.3274]], grad_fn=<MmBackward0>)

正如我们所见，这与我们使用嵌入层时得到的结果完全相同：

In [13]:
embedding(idx)

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]],
       grad_fn=<EmbeddingBackward0>)

- 在底层发生的计算对于第一个训练样本的 token ID 如下：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/4.png" width="450px">

- 对于第二个训练样本的 token ID，计算过程如下：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/5.png" width="450px">

- 由于每个一热编码行中除一个索引外其余均为 0（按设计如此），这种矩阵乘法实际上等同于一热元素的查找。
- 在一热编码上使用矩阵乘法等效于嵌入层查找，但如果处理大型嵌入矩阵时，可能会因为存在大量无用的零乘法而变得低效。